#LGB 0.636 mse
#0.639 mae

# 0.50151896000 lgb

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [54]:
!mkdir -p GuodongDrive
!google-drive-ocamlfuse GuodongDrive
import os
os.chdir("GuodongDrive")
os.chdir("ccf/carsale")
!ls

0825ccf_carsale.csv	   baseline2.ipynb
0825ccf_car_sales_lgb.csv  catboost_info
083101_car_sale.csv	   ccf_car_sales082601.csv
083101sub.csv		   ccf_car_sales082602.csv
083103_car_sale.csv	   ccf_car_sales.csv
083103sub.csv		   ccf_car_sales_lgb.csv
090101car_sales.csv	   evaluation_public.csv
090102car_sales.csv	   evaluation_public_new.csv
090103car_sales.csv	   evaluation_public_xishao.csv
090201car_sales.csv	   guize1-new.ipynb
baseline082601.ipynb	   GuodongDrive
baseline082701.ipynb	   ronghe_car_sales.csv
baseline082702.ipynb	   submit_example.csv
baseline082703.ipynb	   train_sales_data.csv
baseline082704.ipynb	   train_sales_new.csv
baseline083101.ipynb	   train_search_data.csv
baseline083102.ipynb	   train_user_reply_data.csv
baseline090101.ipynb	   Untitled0.ipynb
baseline1_copy.ipynb	   xishao_car_sale.csv
baseline1.ipynb		   统计_car_sales.csv


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
import lightgbm as lgb
import catboost as ctb

import matplotlib.pyplot as plt
import seaborn as sns 

%matplotlib inline 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [2]:
from sklearn import linear_model, svm, gaussian_process
from sklearn.ensemble import RandomForestRegressor

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

思路：  
- 这种popularity	carCommentVolum	newsReplyVolum我感觉都是增长的，可以预测出来,我们可以单独用模型将接下来四个月份的这三个指标预测出来

- 在统计分数的时候所用的验证集，已经作为模型的验证集训练过，所以得到的分数会偏高
应该用split先将train_set分为train和valid，再去用17年最后的四个月作为统计分数的指标

- 可以用KFold进行train_set和valid_set的训练

- 他在训练时将特征model和type换成另一种表现方式，不妨把这种表现方式设为新的特征，model和type依然保留作为特征

-预测流量指数考虑clfs = {
        'svm':svm.SVR(), 
        'RandomForestRegressor':RandomForestRegressor(n_estimators=400),
        'BayesianRidge':linear_model.BayesianRidge()
       }
for clf in clfs:
    try:
        clfs[clf].fit(X_train, y_train)
        y_pred = clfs[clf].predict(X_test)
        print(clf + " cost:" + str(np.sum(y_pred-y_test)/len(y_pred)) )

In [35]:
train_sales_data = pd.read_csv('train_sales_data_train.csv')

train_search_data = pd.read_csv('train_search_data_train.csv')
train_user_reply_data = pd.read_csv('train_user_reply_data_train.csv')
test = pd.read_csv('evaluation_public_9.csv')
del train_sales_data['Unnamed: 0'],train_search_data['Unnamed: 0'],train_user_reply_data['Unnamed: 0']

In [4]:
test.tail()

,province,adcode,model,regYear,regMonth,forecastVolum,id
5275,福建,350000,a9a43d1a7ecbe75d,2017,12,0,5276
5276,辽宁,210000,a9a43d1a7ecbe75d,2017,12,0,5277
5277,重庆,500000,a9a43d1a7ecbe75d,2017,12,0,5278
5278,陕西,610000,a9a43d1a7ecbe75d,2017,12,0,5279
5279,黑龙江,230000,a9a43d1a7ecbe75d,2017,12,0,5280


In [36]:
# data = pd.concat([train_sales_data, test], ignore_index=True)
# data = data.merge(train_search_data, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
# data = data.merge(train_user_reply_data, 'left', on=['model', 'regYear', 'regMonth'])
# data.head(10)

,adcode,bodyType,forecastVolum,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum
0,310000,SUV,NaN,NaN,3c974920a76ac9c1,上海,1,2016,292.0,1479.0,11.0,106.0
1,530000,SUV,NaN,NaN,3c974920a76ac9c1,云南,1,2016,466.0,1594.0,11.0,106.0
2,150000,SUV,NaN,NaN,3c974920a76ac9c1,内蒙古,1,2016,257.0,1479.0,11.0,106.0
3,110000,SUV,NaN,NaN,3c974920a76ac9c1,北京,1,2016,408.0,2370.0,11.0,106.0
4,510000,SUV,NaN,NaN,3c974920a76ac9c1,四川,1,2016,610.0,3562.0,11.0,106.0
5,340000,SUV,NaN,NaN,3c974920a76ac9c1,安徽,1,2016,206.0,1314.0,11.0,106.0
6,370000,SUV,NaN,NaN,3c974920a76ac9c1,山东,1,2016,503.0,3476.0,11.0,106.0
7,140000,SUV,NaN,NaN,3c974920a76ac9c1,山西,1,2016,236.0,1422.0,11.0,106.0
8,440000,SUV,NaN,NaN,3c974920a76ac9c1,广东,1,2016,3635.0,7182.0,11.0,106.0
9,450000,SUV,NaN,NaN,3c974920a76ac9c1,广西,1,2016,450.0,1163.0,11.0,106.0


In [36]:

data = pd.concat([train_sales_data, test], ignore_index=True)
data = data.merge(train_search_data, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user_reply_data, 'left', on=['model', 'regYear', 'regMonth'])
data.head(10)

,adcode,bodyType,forecastVolum,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum
0,310000,SUV,NaN,NaN,3c974920a76ac9c1,上海,1,2016,292.0,1479.0,11.0,106.0
1,530000,SUV,NaN,NaN,3c974920a76ac9c1,云南,1,2016,466.0,1594.0,11.0,106.0
2,150000,SUV,NaN,NaN,3c974920a76ac9c1,内蒙古,1,2016,257.0,1479.0,11.0,106.0
3,110000,SUV,NaN,NaN,3c974920a76ac9c1,北京,1,2016,408.0,2370.0,11.0,106.0
4,510000,SUV,NaN,NaN,3c974920a76ac9c1,四川,1,2016,610.0,3562.0,11.0,106.0
5,340000,SUV,NaN,NaN,3c974920a76ac9c1,安徽,1,2016,206.0,1314.0,11.0,106.0
6,370000,SUV,NaN,NaN,3c974920a76ac9c1,山东,1,2016,503.0,3476.0,11.0,106.0
7,140000,SUV,NaN,NaN,3c974920a76ac9c1,山西,1,2016,236.0,1422.0,11.0,106.0
8,440000,SUV,NaN,NaN,3c974920a76ac9c1,广东,1,2016,3635.0,7182.0,11.0,106.0
9,450000,SUV,NaN,NaN,3c974920a76ac9c1,广西,1,2016,450.0,1163.0,11.0,106.0


In [6]:
data.tail()

,adcode,bodyType,forecastVolum,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum
31675,350000,NaN,0.0,5276.0,a9a43d1a7ecbe75d,福建,12,2017,NaN,NaN,NaN,NaN
31676,210000,NaN,0.0,5277.0,a9a43d1a7ecbe75d,辽宁,12,2017,NaN,NaN,NaN,NaN
31677,500000,NaN,0.0,5278.0,a9a43d1a7ecbe75d,重庆,12,2017,NaN,NaN,NaN,NaN
31678,610000,NaN,0.0,5279.0,a9a43d1a7ecbe75d,陕西,12,2017,NaN,NaN,NaN,NaN
31679,230000,NaN,0.0,5280.0,a9a43d1a7ecbe75d,黑龙江,12,2017,NaN,NaN,NaN,NaN


In [10]:
day_map = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
data['daycount']=data['regMonth'].map(day_map)
data.loc[(data.regMonth==2)&(data.regYear==2016),'daycount']=29

 **每年二月份买车都比较少，感觉应该单独预测**

**每年十二月份比较多，感觉可以去掉**

In [37]:
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
del data['salesVolume'], data['forecastVolum']
data['bodyType_2'] = data['model'].map(train_sales_data.drop_duplicates('model').set_index('model')['bodyType'])
data['model'].head()
#一种model可能对应多种type

0    3c974920a76ac9c1
1    3c974920a76ac9c1
2    3c974920a76ac9c1
3    3c974920a76ac9c1
4    3c974920a76ac9c1
Name: model, dtype: object

In [38]:
data['model_2'] = data['model']

In [9]:
data.head(10)

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,bodyType_2,model_2
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,1479.0,11.0,106.0,292.0,SUV,3c974920a76ac9c1
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,1594.0,11.0,106.0,466.0,SUV,3c974920a76ac9c1
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,1479.0,11.0,106.0,257.0,SUV,3c974920a76ac9c1
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,2370.0,11.0,106.0,408.0,SUV,3c974920a76ac9c1
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,3562.0,11.0,106.0,610.0,SUV,3c974920a76ac9c1
5,340000,SUV,0,3c974920a76ac9c1,安徽,1,2016,1314.0,11.0,106.0,206.0,SUV,3c974920a76ac9c1
6,370000,SUV,0,3c974920a76ac9c1,山东,1,2016,3476.0,11.0,106.0,503.0,SUV,3c974920a76ac9c1
7,140000,SUV,0,3c974920a76ac9c1,山西,1,2016,1422.0,11.0,106.0,236.0,SUV,3c974920a76ac9c1
8,440000,SUV,0,3c974920a76ac9c1,广东,1,2016,7182.0,11.0,106.0,3635.0,SUV,3c974920a76ac9c1
9,450000,SUV,0,3c974920a76ac9c1,广西,1,2016,1163.0,11.0,106.0,450.0,SUV,3c974920a76ac9c1


In [10]:
data.tail(10)

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,bodyType_2,model_2
31670,130000,NaN,5271,a9a43d1a7ecbe75d,河北,12,2017,NaN,NaN,NaN,NaN,SUV,a9a43d1a7ecbe75d
31671,410000,NaN,5272,a9a43d1a7ecbe75d,河南,12,2017,NaN,NaN,NaN,NaN,SUV,a9a43d1a7ecbe75d
31672,330000,NaN,5273,a9a43d1a7ecbe75d,浙江,12,2017,NaN,NaN,NaN,NaN,SUV,a9a43d1a7ecbe75d
31673,420000,NaN,5274,a9a43d1a7ecbe75d,湖北,12,2017,NaN,NaN,NaN,NaN,SUV,a9a43d1a7ecbe75d
31674,430000,NaN,5275,a9a43d1a7ecbe75d,湖南,12,2017,NaN,NaN,NaN,NaN,SUV,a9a43d1a7ecbe75d
31675,350000,NaN,5276,a9a43d1a7ecbe75d,福建,12,2017,NaN,NaN,NaN,NaN,SUV,a9a43d1a7ecbe75d
31676,210000,NaN,5277,a9a43d1a7ecbe75d,辽宁,12,2017,NaN,NaN,NaN,NaN,SUV,a9a43d1a7ecbe75d
31677,500000,NaN,5278,a9a43d1a7ecbe75d,重庆,12,2017,NaN,NaN,NaN,NaN,SUV,a9a43d1a7ecbe75d
31678,610000,NaN,5279,a9a43d1a7ecbe75d,陕西,12,2017,NaN,NaN,NaN,NaN,SUV,a9a43d1a7ecbe75d
31679,230000,NaN,5280,a9a43d1a7ecbe75d,黑龙江,12,2017,NaN,NaN,NaN,NaN,SUV,a9a43d1a7ecbe75d


In [11]:
range(data['model_2'].nunique())

range(0, 60)

In [39]:
for i in ['bodyType_2', 'model_2']:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))#用这个函数可以查看数据有多少个不同值。

data.head(10)

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,bodyType_2,model_2
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,1479.0,11.0,106.0,292.0,0,0
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,1594.0,11.0,106.0,466.0,0,0
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,1479.0,11.0,106.0,257.0,0,0
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,2370.0,11.0,106.0,408.0,0,0
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,3562.0,11.0,106.0,610.0,0,0
5,340000,SUV,0,3c974920a76ac9c1,安徽,1,2016,1314.0,11.0,106.0,206.0,0,0
6,370000,SUV,0,3c974920a76ac9c1,山东,1,2016,3476.0,11.0,106.0,503.0,0,0
7,140000,SUV,0,3c974920a76ac9c1,山西,1,2016,1422.0,11.0,106.0,236.0,0,0
8,440000,SUV,0,3c974920a76ac9c1,广东,1,2016,7182.0,11.0,106.0,3635.0,0,0
9,450000,SUV,0,3c974920a76ac9c1,广西,1,2016,1163.0,11.0,106.0,450.0,0,0


zip() 函数用于将可迭代的对象作为参数，

将对象中对应的元素打包成一个个元组，

然后返回由这些元组组成的列表。

如果各个迭代器的元素个数不一致，则返回列表长度与最短的对象相同，

a = [1,2,3]

 b = [4,5,6]

 c = [4,5,6,7,8]

 zipped = zip(a,b)  # 打包为元组的列表

[(1, 4), (2, 5), (3, 6)]

zip(a,c)              # 元素个数与最短的列表一致

[(1, 4), (2, 5), (3, 6)]

In [40]:
data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']

# 'popularity', 'carCommentVolum', 'newsReplyVolum','label'/

shift_feat = []

data['model_adcode'] = data['adcode'] + data['model_2']
data['model_adcode_mt'] = data['model_adcode'] * 100 + data['mt']
for i in [11]:
    i = i + 1
    shift_feat.append('shift_model_adcode_mt_label_{0}'.format(i))
    data['model_adcode_mt_{0}'.format(i)] = data['model_adcode_mt'] + i
    data_last = data[~data.label.isnull()].set_index('model_adcode_mt_{0}'.format(i))
    data['shift_model_adcode_mt_label_{0}'.format(i)] = data['model_adcode_mt'].map(data_last['label'])

shift_feat

['shift_model_adcode_mt_label_12']

In [14]:
data.head(10)

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,bodyType_2,model_2,mt,model_adcode,model_adcode_mt,model_adcode_mt_12,shift_model_adcode_mt_label_12
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,1479.0,11.0,106.0,292.0,0,0,1,310000,31000001,31000013,NaN
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,1594.0,11.0,106.0,466.0,0,0,1,530000,53000001,53000013,NaN
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,1479.0,11.0,106.0,257.0,0,0,1,150000,15000001,15000013,NaN
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,2370.0,11.0,106.0,408.0,0,0,1,110000,11000001,11000013,NaN
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,3562.0,11.0,106.0,610.0,0,0,1,510000,51000001,51000013,NaN
5,340000,SUV,0,3c974920a76ac9c1,安徽,1,2016,1314.0,11.0,106.0,206.0,0,0,1,340000,34000001,34000013,NaN
6,370000,SUV,0,3c974920a76ac9c1,山东,1,2016,3476.0,11.0,106.0,503.0,0,0,1,370000,37000001,37000013,NaN
7,140000,SUV,0,3c974920a76ac9c1,山西,1,2016,1422.0,11.0,106.0,236.0,0,0,1,140000,14000001,14000013,NaN
8,440000,SUV,0,3c974920a76ac9c1,广东,1,2016,7182.0,11.0,106.0,3635.0,0,0,1,440000,44000001,44000013,NaN
9,450000,SUV,0,3c974920a76ac9c1,广西,1,2016,1163.0,11.0,106.0,450.0,0,0,1,450000,45000001,45000013,NaN


In [15]:
data.tail(10)

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,bodyType_2,model_2,mt,model_adcode,model_adcode_mt,model_adcode_mt_12,shift_model_adcode_mt_label_12
31670,130000,NaN,5271,a9a43d1a7ecbe75d,河北,12,2017,NaN,NaN,NaN,NaN,0,59,24,130059,13005924,13005936,95.0
31671,410000,NaN,5272,a9a43d1a7ecbe75d,河南,12,2017,NaN,NaN,NaN,NaN,0,59,24,410059,41005924,41005936,164.0
31672,330000,NaN,5273,a9a43d1a7ecbe75d,浙江,12,2017,NaN,NaN,NaN,NaN,0,59,24,330059,33005924,33005936,397.0
31673,420000,NaN,5274,a9a43d1a7ecbe75d,湖北,12,2017,NaN,NaN,NaN,NaN,0,59,24,420059,42005924,42005936,236.0
31674,430000,NaN,5275,a9a43d1a7ecbe75d,湖南,12,2017,NaN,NaN,NaN,NaN,0,59,24,430059,43005924,43005936,445.0
31675,350000,NaN,5276,a9a43d1a7ecbe75d,福建,12,2017,NaN,NaN,NaN,NaN,0,59,24,350059,35005924,35005936,99.0
31676,210000,NaN,5277,a9a43d1a7ecbe75d,辽宁,12,2017,NaN,NaN,NaN,NaN,0,59,24,210059,21005924,21005936,155.0
31677,500000,NaN,5278,a9a43d1a7ecbe75d,重庆,12,2017,NaN,NaN,NaN,NaN,0,59,24,500059,50005924,50005936,214.0
31678,610000,NaN,5279,a9a43d1a7ecbe75d,陕西,12,2017,NaN,NaN,NaN,NaN,0,59,24,610059,61005924,61005936,263.0
31679,230000,NaN,5280,a9a43d1a7ecbe75d,黑龙江,12,2017,NaN,NaN,NaN,NaN,0,59,24,230059,23005924,23005936,57.0


In [41]:
num_feat = ['adcode', 'regMonth', 'regYear', 'popularity', 'carCommentVolum', 'newsReplyVolum', 'bodyType_2', 'model_2']+ shift_feat
cate_feat = ['bodyType', 'model', 'province']

# lbl = LabelEncoder()
# for i in cate_feat:
#   data[i] = lbl.fit_transform(data[i].astype(str))
for i in cate_feat:
    data[i] = data[i].astype('category')
features = num_feat + cate_feat

**划分训练集:train验证集

2017.5.6.7.8测试集index**

In [42]:
# data['n_label'] = data['label'] / data.groupby('model')['label'].transform('mean')
train_idx = (data['mt'] <= 16)

valid_idx = (data['mt'].between(17, 20))

test_idx = (data['mt'] > 20)

In [43]:
data['model_weight'] = data.groupby('model_2')['label'].transform('mean')
data['n_label'] = data['label'] / data['model_weight']

In [25]:
data.head(10)

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,bodyType_2,model_2,mt,model_adcode,model_adcode_mt,model_adcode_mt_12,shift_model_adcode_mt_label_12,model_weight,n_label
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,1479.0,11.0,106.0,292.0,0,0,1,310000,31000001,31000013,NaN,441.722727,0.661048
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,1594.0,11.0,106.0,466.0,0,0,1,530000,53000001,53000013,NaN,441.722727,1.054960
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,1479.0,11.0,106.0,257.0,0,0,1,150000,15000001,15000013,NaN,441.722727,0.581813
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,2370.0,11.0,106.0,408.0,0,0,1,110000,11000001,11000013,NaN,441.722727,0.923656
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,3562.0,11.0,106.0,610.0,0,0,1,510000,51000001,51000013,NaN,441.722727,1.380957
5,340000,SUV,0,3c974920a76ac9c1,安徽,1,2016,1314.0,11.0,106.0,206.0,0,0,1,340000,34000001,34000013,NaN,441.722727,0.466356
6,370000,SUV,0,3c974920a76ac9c1,山东,1,2016,3476.0,11.0,106.0,503.0,0,0,1,370000,37000001,37000013,NaN,441.722727,1.138723
7,140000,SUV,0,3c974920a76ac9c1,山西,1,2016,1422.0,11.0,106.0,236.0,0,0,1,140000,14000001,14000013,NaN,441.722727,0.534272
8,440000,SUV,0,3c974920a76ac9c1,广东,1,2016,7182.0,11.0,106.0,3635.0,0,0,1,440000,44000001,44000013,NaN,441.722727,8.229144
9,450000,SUV,0,3c974920a76ac9c1,广西,1,2016,1163.0,11.0,106.0,450.0,0,0,1,450000,45000001,45000013,NaN,441.722727,1.018739


In [44]:
train_x = data[train_idx][features]
train_y = data[train_idx]['n_label']
print("train_x is\n")
#print(train_x)
print("-----------------------------------")
valid_x = data[valid_idx][features]
valid_y = data[valid_idx]['n_label']
print("valid_x is\n")
#print(valid_x)
print("-----------------------------------")

train_x is

-----------------------------------
valid_x is

-----------------------------------


In [45]:
# train_x = data[train_idx][features]
# train_y = data[train_idx]['n_label']
# print("train_x is\n")
# print(train_x)
# print("-----------------------------------")
# valid_x = data[valid_idx][features]
# valid_y = data[valid_idx]['n_label']
# print("valid_x is\n")
# print(valid_x)
# print("-----------------------------------")



lgb_model = lgb.LGBMRegressor(
    num_leaves=64, reg_alpha=1, reg_lambda=0.1, objective='mae',
    max_depth=-1, learning_rate=0.05, min_child_samples=20, random_state=np.random.randint(1000),
    n_estimators=6000, subsample=0.7, colsample_bytree=0.7,n_jobs=-1
)

lgb_model.fit(train_x, train_y, eval_set=[
    (valid_x, valid_y),
], categorical_feature=cate_feat, early_stopping_rounds=100, verbose=100)

#也就是说我们在训练集和验证集上训练完了之后再让它重新预测整个数据集，让其中的valid和vaild



Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 0.240096
[200]	valid_0's l1: 0.208108
[300]	valid_0's l1: 0.193765
[400]	valid_0's l1: 0.188834
[500]	valid_0's l1: 0.186227
[600]	valid_0's l1: 0.184397
[700]	valid_0's l1: 0.18298
[800]	valid_0's l1: 0.182593
[900]	valid_0's l1: 0.182322
[1000]	valid_0's l1: 0.181735
[1100]	valid_0's l1: 0.181218
[1200]	valid_0's l1: 0.180414
[1300]	valid_0's l1: 0.180151
[1400]	valid_0's l1: 0.180075
[1500]	valid_0's l1: 0.179834
[1600]	valid_0's l1: 0.179899
Early stopping, best iteration is:
[1503]	valid_0's l1: 0.179824


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=6000, n_jobs=-1, num_leaves=64, objective='mae',
              random_state=930, reg_alpha=1, reg_lambda=0.1, silent=True,
              subsample=0.7, subsample_for_bin=200000, subsample_freq=0)

In [46]:
data['pred_label'] = lgb_model.predict(data[features]) * data['model_weight']
def valid_train_score(data, pred='pred_label', label='label', group='model'):
  
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data.groupby('model').agg({
        pred: list,
        label: [list, 'mean'],

    }).reset_index()

    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)



best_score = valid_train_score(data[valid_idx])




0.6892350301333952


In [50]:
lgb_model.n_estimators = 6000
test = data[~test_idx][features]
lgb_model.fit(data[~test_idx][features], data[~test_idx]['n_label'], categorical_feature=cate_feat)

data['forecastVolum'] = lgb_model.predict(data[features])   * data['model_weight']
sub = data[test_idx][['id']]
sub['forecastVolum'] = data[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
sub.head(10)
#sub.to_csv('082703sub.csv', index=False)

,id,forecastVolum
26400,1,147
26401,2,171
26402,3,104
26403,4,182
26404,5,285
26405,6,116
26406,7,324
26407,8,123
26408,9,1657
26409,10,166


In [57]:
data[test_idx].head()

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,bodyType_2,model_2,mt,model_adcode,model_adcode_mt,model_adcode_mt_12,shift_model_adcode_mt_label_12,model_weight,n_label,pred_label,forecastVolum
26400,310000,NaN,1,3c974920a76ac9c1,上海,9,2017,NaN,NaN,NaN,NaN,0,0,21,310000,31000021,31000033,265.0,441.722727,NaN,130.492103,146.531924
26401,530000,NaN,2,3c974920a76ac9c1,云南,9,2017,NaN,NaN,NaN,NaN,0,0,21,530000,53000021,53000033,217.0,441.722727,NaN,122.715117,170.550898
26402,150000,NaN,3,3c974920a76ac9c1,内蒙古,9,2017,NaN,NaN,NaN,NaN,0,0,21,150000,15000021,15000033,157.0,441.722727,NaN,71.341518,103.617418
26403,110000,NaN,4,3c974920a76ac9c1,北京,9,2017,NaN,NaN,NaN,NaN,0,0,21,110000,11000021,11000033,337.0,441.722727,NaN,202.240237,181.942058
26404,510000,NaN,5,3c974920a76ac9c1,四川,9,2017,NaN,NaN,NaN,NaN,0,0,21,510000,51000021,51000033,623.0,441.722727,NaN,190.511049,285.386091


In [60]:
# #用规则测试一下分数
# sub3 = pd.read_csv('valid_data.csv')
# sub3.head()
# # data[test_idx]['forecastVolum'] = sub3['forecastVolum']
# data2=data[test_idx]
# data2['forecastVolum'] = sub3['forecastVolum'].values
# data2.head()

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,bodyType_2,model_2,mt,model_adcode,model_adcode_mt,model_adcode_mt_12,shift_model_adcode_mt_label_12,model_weight,n_label,pred_label,forecastVolum
26400,310000,NaN,1,3c974920a76ac9c1,上海,9,2017,NaN,NaN,NaN,NaN,0,0,21,310000,31000021,31000033,265.0,441.722727,NaN,130.492103,373
26401,530000,NaN,2,3c974920a76ac9c1,云南,9,2017,NaN,NaN,NaN,NaN,0,0,21,530000,53000021,53000033,217.0,441.722727,NaN,122.715117,240
26402,150000,NaN,3,3c974920a76ac9c1,内蒙古,9,2017,NaN,NaN,NaN,NaN,0,0,21,150000,15000021,15000033,157.0,441.722727,NaN,71.341518,166
26403,110000,NaN,4,3c974920a76ac9c1,北京,9,2017,NaN,NaN,NaN,NaN,0,0,21,110000,11000021,11000033,337.0,441.722727,NaN,202.240237,380
26404,510000,NaN,5,3c974920a76ac9c1,四川,9,2017,NaN,NaN,NaN,NaN,0,0,21,510000,51000021,51000033,623.0,441.722727,NaN,190.511049,590


In [61]:
def valid_data_score(data2):
  
    #data2 = data[test_idx]
    sub_y = pd.read_csv('valid_data_y.csv')
    data2['label'] = sub_y['salesVolume'].values
    data2['pred_label'] = data2['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data2.groupby('model').agg({
            'pred_label': list,
            'label': [list, 'mean'],

        }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['pred_label_list', 'label_list', 'label_mean']].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)



best_score = valid_data_score(data2)


0.6577652508918383


In [63]:
data.head()

,adcode,bodyType,bodyType_2,carCommentVolum,forecastVolum,id,label,model,model_2,model_adcode,model_adcode_mt,model_adcode_mt_12,model_weight,mt,n_label,newsReplyVolum,popularity,pred_label,predict_label,province,regMonth,regYear,sample_weight,shift_model_adcode_mt_label_12
0,310000,SUV,0,11.0,312.597092,0,292.0,3c974920a76ac9c1,0,310000,31000001,31000013,441.722727,1,0.661048,106.0,1479.0,318.405115,571.612137,上海,1,2016,1,NaN
1,530000,SUV,0,11.0,416.063528,0,466.0,3c974920a76ac9c1,0,530000,53000001,53000013,441.722727,1,1.054960,106.0,1594.0,429.624473,561.534158,云南,1,2016,1,NaN
2,150000,SUV,0,11.0,255.496750,0,257.0,3c974920a76ac9c1,0,150000,15000001,15000013,441.722727,1,0.581813,106.0,1479.0,244.092446,576.782267,内蒙古,1,2016,1,NaN
3,110000,SUV,0,11.0,408.113294,0,408.0,3c974920a76ac9c1,0,110000,11000001,11000013,441.722727,1,0.923656,106.0,2370.0,440.485346,588.089368,北京,1,2016,1,NaN
4,510000,SUV,0,11.0,609.962454,0,610.0,3c974920a76ac9c1,0,510000,51000001,51000013,441.722727,1,1.380957,106.0,3562.0,571.260206,584.663677,四川,1,2016,1,NaN


In [23]:
sub.to_csv('0903car_sales.csv', index=False)

In [24]:
evaluation_public_1 = pd.read_csv("rongheguize_0902.csv")
evaluation_public_1.head(5)

,id,forecastVolum
0,1,288
1,2,376
2,3,154
3,4,332
4,5,402


In [0]:
evaluation_public_1.shape

(5280, 2)

In [25]:
evaluation_public_2 = pd.read_csv("0903car_sales.csv")

evaluation_public_2.head(5)

,id,forecastVolum
0,1,336
1,2,414
2,3,172
3,4,420
4,5,436


In [26]:
evaluation_public_2.shape

(5280, 2)

In [27]:
evaluation_public_2['forecastVolum'] = evaluation_public_2['forecastVolum']*0.3 + evaluation_public_1['forecastVolum']*0.7
evaluation_public_2.head()

,id,forecastVolum
0,1,302.4
1,2,387.4
2,3,159.4
3,4,358.4
4,5,412.2


In [28]:
evaluation_public_2['forecastVolum'] = evaluation_public_2['forecastVolum'].round().astype(int)
evaluation_public_2.head()

,id,forecastVolum
0,1,302
1,2,387
2,3,159
3,4,358
4,5,412


In [30]:
evaluation_public_2.to_csv('ronghe0903_car_sale.csv', index=False)